In [ ]:

#  Clean install for Kaggle-compatible environment

# Uninstall potentially conflicting packages
!pip uninstall -y bitsandbytes triton unsloth unsloth-zoo transformers peft accelerate trl sentence-transformers

# Reinstall compatible versions (tested to work on Kaggle as of June 2025)
!pip install -q bitsandbytes==0.41.1
!pip install -q transformers==4.40.2
!pip install -q peft==0.10.0
!pip install -q accelerate==0.30.0
!pip install -q trl==0.7.10
!pip install -q sentence-transformers==2.6.1

# Install Unsloth without Triton-related optimization
!pip install -q git+https://github.com/unslothai/unsloth.git

# Verify installation
import torch, transformers, peft, accelerate, bitsandbytes, unsloth

print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("accelerate:", accelerate.__version__)
print("bitsandbytes:", bitsandbytes.__version__)
print("unsloth installed successfully!")


In [ ]:
print(" Kernel is responding")


In [ ]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"


In [ ]:
rm -rf ~/.cache/triton


In [ ]:
from IPython.display import display
import ipywidgets as widgets

uploader = widgets.FileUpload(accept='.jsonl', multiple=False)
display(uploader)


In [ ]:

!pip install bitsandbytes




In [ ]:
import bitsandbytes as bnb
print(" bitsandbytes is working!")


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  
load_in_4bit = True    
dtype = None           

# Available 4-bit models
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",  # Just change this string to pick another model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    ##token = "hf_..."  # Replace with your Hugging Face token if needed
)


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device")


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Choose any number > 0! Suggested: 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,

)


In [ ]:
from IPython.display import display
import ipywidgets as widgets

uploader = widgets.FileUpload(accept='.jsonl', multiple=False)
display(uploader)


In [ ]:
import io

for fileinfo in uploader.value:
    filename = fileinfo['name']
    content = fileinfo['content']
    with open(f"/kaggle/working/{filename}", "wb") as f:
        f.write(content)
    print(f"Saved: {filename}")


In [ ]:
with open("/kaggle/working/pid_llm_data.jsonl", "r") as f:
    data = f.readlines()

print(data[:3])


In [ ]:
import os

print("Files in /kaggle/working/:")
print(os.listdir("/kaggle/working/"))


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="pid_llm_data.jsonl")["train"]


In [ ]:
print(dataset[0])


In [ ]:
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer


In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(example):
    messages = example.get("messages", [])
    text = ""

    # Ensure messages is a flat list of dicts
    for msg in messages:
        # If msg is a nested list (bad format), skip or flatten
        if isinstance(msg, list):
            msg_list = msg
        else:
            msg_list = [msg]

        for submsg in msg_list:
            if isinstance(submsg, dict):
                role = submsg.get("role", "").lower()
                content = submsg.get("content", "")
                if role == "user":
                    text += f"User: {content}\n"
                elif role == "assistant":
                    text += f"Assistant: {content}\n"

    # Append EOS token only if text exists
    return [text.strip() + EOS_TOKEN if text.strip() else EOS_TOKEN]


In [ ]:
def tokenize(example):
    texts = formatting_func(example)
    return tokenizer(
        texts[0],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize)


In [ ]:
def tokenize(example):
    texts = formatting_func(example)
    return tokenizer(
        texts[0],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors=None
    )

tokenized_dataset = dataset.map(tokenize)


In [ ]:
#pip install --upgrade transformers


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    evaluation_strategy="steps",           
    eval_steps=100,                        
    load_best_model_at_end=True,           
    metric_for_best_model="eval_loss",     
    greater_is_better=False,               
    report_to="none",
)


In [ ]:

from transformers import EarlyStoppingCallback

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=512,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
# Stops after 2 non-improving evals
)


In [ ]:
#!pip uninstall triton -y
#!pip install --upgrade triton


In [ ]:
!rm -rf ~/.cache/triton


In [ ]:
trainer_stats = trainer.train()

In [ ]:
print(trainer.args)
print("Training enabled?", trainer.args.do_train)
